In [1]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb

In [2]:
train = pd.read_excel('QA_from_val.xlsx')
train.head()

,Unnamed: 0,Logeq,Mutex,Necc,answer,qid,question
0,0,is this dude using knife to cut the cake?,is this dude using a hammer to cut the cake?,is there any cake in the picture?,knife,340069003,This bored dude is using what instrument to cu...
1,3,is a city bus pictured?,is the type of bus pictured a tourist bus?,is there a bus pictured?,city bus,188817001,What type of bus is pictured?
2,8,are there logs on the ground behind the giraffe?,is there spoon laying on the ground behind the...,is there a giraffe?,logs,573778003,What is laying on the ground behind the giraffe?
3,10,is the horse brown?,is the horse black?,is there a horse?,brown,405135001,What color is the horse?
4,11,is the surfboard green?,is the surfboard yellow?,is there a surfboard?,green,187362006,What color is the surfboard?


In [3]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [4]:
trainset=[]
for i in range(0,len(train['question'])):
    trainset.append(str(train['question'][i])+" "+str(train['answer'][i]))
train['input']=trainset

In [5]:
print(train['input'])

0        This bored dude is using what instrument to cu...
1                   What type of bus is pictured? city bus
2        What is laying on the ground behind the giraff...
3                           What color is the horse? brown
4                       What color is the surfboard? green
                               ...                        
10316             What is the man talking into? microphone
10317    What is in between the trees and the parking s...
10318    What is the gray, white, and black item in the...
10319                   How many birds are on the posts? 4
10320        What is the guy wearing on his back? backpack
Name: input, Length: 10321, dtype: object


In [6]:
tokenized = train['input'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [7]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [8]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(10321, 24)

In [9]:
input_ids = torch.tensor(padded)  
for i in range(len(input_ids)):
    for j in range(len(input_ids[i])):
        input_ids[i][j]=int(input_ids[i][j])
attention_mask = torch.tensor(attention_mask)

In [10]:
input_ids = torch.tensor(input_ids).to(torch.int64)

C:\Users\Manish\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [11]:
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [13]:
print(last_hidden_states[0])

tensor([[[ 0.0928,  0.0945, -0.0275,  ..., -0.1033,  0.5134,  0.1610],
         [ 0.0147,  0.2024,  0.2038,  ..., -0.3736,  0.7296, -0.1233],
         [ 0.6360,  0.4243,  0.2931,  ..., -0.0370,  0.2403, -0.4834],
         ...,
         [ 0.1832,  0.1939,  0.0443,  ..., -0.0902,  0.0797, -0.1447],
         [ 0.2112,  0.1484, -0.0090,  ..., -0.0316,  0.0862, -0.1411],
         [ 0.1483,  0.0837, -0.0177,  ...,  0.0605,  0.0572, -0.0915]],

        [[-0.0354, -0.1693,  0.1140,  ..., -0.1447,  0.4527,  0.2520],
         [-0.1267, -0.4382,  0.2802,  ...,  0.1191,  0.4038,  0.1366],
         [ 0.3612, -0.3415,  0.5272,  ..., -0.4015,  0.5448,  0.0301],
         ...,
         [ 0.1055, -0.2255,  0.4025,  ..., -0.0354,  0.2002, -0.0845],
         [ 0.2109, -0.2350,  0.4623,  ..., -0.0258,  0.1898, -0.0359],
         [ 0.1941, -0.3391,  0.1602,  ...,  0.0799, -0.0240, -0.0214]],

        [[-0.0103,  0.0982, -0.1870,  ..., -0.0775,  0.4835,  0.0642],
         [ 0.0312,  0.0033, -0.2888,  ...,  0

In [14]:
total_length=len(last_hidden_states[0])
test_length=2*total_length//3
print(total_length)
print(test_length)

10321
6880


In [15]:
test_input=last_hidden_states[0][:test_length]
val_input=last_hidden_states[0][test_length:]
print(test_input)
print()
print(val_input)

tensor([[[ 0.0928,  0.0945, -0.0275,  ..., -0.1033,  0.5134,  0.1610],
         [ 0.0147,  0.2024,  0.2038,  ..., -0.3736,  0.7296, -0.1233],
         [ 0.6360,  0.4243,  0.2931,  ..., -0.0370,  0.2403, -0.4834],
         ...,
         [ 0.1832,  0.1939,  0.0443,  ..., -0.0902,  0.0797, -0.1447],
         [ 0.2112,  0.1484, -0.0090,  ..., -0.0316,  0.0862, -0.1411],
         [ 0.1483,  0.0837, -0.0177,  ...,  0.0605,  0.0572, -0.0915]],

        [[-0.0354, -0.1693,  0.1140,  ..., -0.1447,  0.4527,  0.2520],
         [-0.1267, -0.4382,  0.2802,  ...,  0.1191,  0.4038,  0.1366],
         [ 0.3612, -0.3415,  0.5272,  ..., -0.4015,  0.5448,  0.0301],
         ...,
         [ 0.1055, -0.2255,  0.4025,  ..., -0.0354,  0.2002, -0.0845],
         [ 0.2109, -0.2350,  0.4623,  ..., -0.0258,  0.1898, -0.0359],
         [ 0.1941, -0.3391,  0.1602,  ...,  0.0799, -0.0240, -0.0214]],

        [[-0.0103,  0.0982, -0.1870,  ..., -0.0775,  0.4835,  0.0642],
         [ 0.0312,  0.0033, -0.2888,  ...,  0

In [16]:
test_output_sentences=train['Logeq'][:test_length]
val_output_sentences=train['Logeq'][test_length:]
print(test_output_sentences)
print(val_output_sentences)

0              is this dude using knife to cut the cake?
1                                is a city bus pictured?
2       are there logs on the ground behind the giraffe?
3                                    is the horse brown?
4                                is the surfboard green?
                              ...                       
6875                              are there 12 cupcakes?
6876                       is she wearing a white shirt?
6877                    is that heineken  brand of beer?
6878                  was this photo taken in starbucks?
6879                                are there 3 glasses?
Name: Logeq, Length: 6880, dtype: object
6880                        is the man using a skateboard?
6881              is the hydrant smiling because graffiti?
6882                     is the monitor sitting on a desk?
6883                                     is the dog black?
6884                                 are 4 horses present?
                               ...   

In [17]:
import tensorflow as tf
from tensorflow import keras

In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [19]:
import tensorflow.keras.utils as ku 
import numpy as np 

In [20]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np 

In [21]:
tokenizer = Tokenizer()

In [22]:
answrs=[]
for i in range(total_length):
    answrs.append("qwertyuiop " + str(train['Logeq'][i]))
tokenizer.fit_on_texts(answrs)

In [23]:
total_words = len(tokenizer.word_index) + 1

In [24]:
total_words

3434

In [25]:
tokenizer.word_index['qwertyuiop']

1

In [26]:
answer_seq=tokenizer.texts_to_sequences(answrs)

In [27]:
answer_seq

[[1, 3, 9, 1967, 203, 397, 17, 415, 2, 105],
 [1, 3, 5, 416, 41, 147],
 [1, 4, 6, 1464, 7, 2, 90, 91, 2, 97],
 [1, 3, 2, 73, 28],
 [1, 3, 2, 128, 25],
 [1, 2, 586, 2, 33, 3, 22, 159, 519],
 [1, 3, 2, 234, 161, 28],
 [1, 3, 55, 40, 4, 22, 20],
 [1, 3, 9, 5, 73],
 [1, 3, 9, 234, 555, 23],
 [1, 3, 2, 496, 102, 415],
 [1, 3, 2, 1968, 10, 9, 31, 5, 1465],
 [1, 3, 9, 1969, 352, 10, 294],
 [1, 4, 6, 417, 275, 100],
 [1, 3, 2, 53, 690, 19],
 [1, 4, 6, 324, 1466, 587, 8, 9, 46],
 [1, 3, 2, 171, 7, 30, 777],
 [1, 3, 6, 5, 374, 10, 176, 6],
 [1, 3, 9, 53, 418, 23],
 [1, 3, 2, 55, 125, 2, 67, 27, 7, 226, 691, 28, 34, 12],
 [1, 3, 2, 55, 10, 2, 122, 28],
 [1, 4, 520, 521, 7, 2, 692, 133, 17, 2, 119],
 [1, 3, 2, 87, 32, 72],
 [1, 3, 2, 172, 7, 2, 693, 12],
 [1, 3, 2, 260, 353, 987, 2, 116, 10, 2, 276, 398],
 [1, 3, 5, 154, 522],
 [1, 3, 9, 21, 22, 5, 20, 1970],
 [1, 3, 9, 523, 634],
 [1, 3, 2, 55, 10, 2, 462, 2, 11, 24, 2, 72, 3, 22, 12],
 [1, 3, 2, 11, 24, 5, 106, 8, 30, 59],
 [1, 3, 2, 128, 2, 113

In [28]:
max_len = 0
for i in answer_seq:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in answer_seq])

In [29]:
padded

array([[  1,   3,   9, ...,   0,   0,   0],
       [  1,   3,   5, ...,   0,   0,   0],
       [  1,   4,   6, ...,   0,   0,   0],
       ...,
       [  1,   3,   2, ..., 544,   0,   0],
       [  1,   4,  45, ...,   0,   0,   0],
       [  1,   3,   2, ...,   0,   0,   0]])

In [30]:
index_to_word=dict([(i+1,char) for (i,char) in enumerate(tokenizer.word_index)])

In [31]:
print(tokenizer.word_index['is'])
print(index_to_word)

3
{1: 'qwertyuiop', 2: 'the', 3: 'is', 4: 'are', 5: 'a', 6: 'there', 7: 'on', 8: 'in', 9: 'this', 10: 'of', 11: 'man', 12: 'white', 13: '2', 14: '1', 15: 'people', 16: 'does', 17: 'to', 18: 'picture', 19: 'red', 20: 'black', 21: 'person', 22: 'wearing', 23: 'blue', 24: 'holding', 25: 'green', 26: '3', 27: 'have', 28: 'brown', 29: 'cat', 30: 'his', 31: 'dog', 32: 'playing', 33: 'woman', 34: 'and', 35: 'shirt', 36: 'left', 37: 'right', 38: 'photo', 39: 'these', 40: 'they', 41: 'bus', 42: 'made', 43: 'sign', 44: 'yellow', 45: '4', 46: 'train', 47: 'at', 48: 'do', 49: 'sitting', 50: 'table', 51: 'water', 52: 'pizza', 53: "man's", 54: 'for', 55: 'color', 56: 'it', 57: 'you', 58: 'standing', 59: 'hand', 60: 'shown', 61: 'plate', 62: 'animal', 63: 'clock', 64: '0', 65: 'an', 66: 'tennis', 67: 'animals', 68: 'food', 69: 'wall', 70: 'orange', 71: 'room', 72: 'frisbee', 73: 'horse', 74: 'top', 75: 'background', 76: 'can', 77: 'with', 78: 'boy', 79: 'number', 80: 'toilet', 81: '5', 82: 'eating', 

In [32]:
index_to_word[627]

'belong'

In [33]:
tokenizer.word_index

{'qwertyuiop': 1,
 'the': 2,
 'is': 3,
 'are': 4,
 'a': 5,
 'there': 6,
 'on': 7,
 'in': 8,
 'this': 9,
 'of': 10,
 'man': 11,
 'white': 12,
 '2': 13,
 '1': 14,
 'people': 15,
 'does': 16,
 'to': 17,
 'picture': 18,
 'red': 19,
 'black': 20,
 'person': 21,
 'wearing': 22,
 'blue': 23,
 'holding': 24,
 'green': 25,
 '3': 26,
 'have': 27,
 'brown': 28,
 'cat': 29,
 'his': 30,
 'dog': 31,
 'playing': 32,
 'woman': 33,
 'and': 34,
 'shirt': 35,
 'left': 36,
 'right': 37,
 'photo': 38,
 'these': 39,
 'they': 40,
 'bus': 41,
 'made': 42,
 'sign': 43,
 'yellow': 44,
 '4': 45,
 'train': 46,
 'at': 47,
 'do': 48,
 'sitting': 49,
 'table': 50,
 'water': 51,
 'pizza': 52,
 "man's": 53,
 'for': 54,
 'color': 55,
 'it': 56,
 'you': 57,
 'standing': 58,
 'hand': 59,
 'shown': 60,
 'plate': 61,
 'animal': 62,
 'clock': 63,
 '0': 64,
 'an': 65,
 'tennis': 66,
 'animals': 67,
 'food': 68,
 'wall': 69,
 'orange': 70,
 'room': 71,
 'frisbee': 72,
 'horse': 73,
 'top': 74,
 'background': 75,
 'can': 76,
 

In [34]:
decoder_input_data = np.zeros(
    (len(padded), max_len, total_words),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(padded), max_len, total_words),
    dtype='float32')

batch_size = 64
epochs = 100
latent_dim = 200
num_samples = test_input.shape[0]

print(test_input.shape)

torch.Size([6880, 24, 768])


In [35]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [36]:
for i, seq in enumerate(padded):
    for t, num in enumerate(seq):
       
        decoder_input_data[i, t, num] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, num] = 1.

encoder_inputs = Input(shape=(None, test_input.shape[2]))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

In [37]:
print(decoder_input_data.shape)

(10321, 19, 3434)


In [38]:
decoder_input_data=decoder_input_data[:test_length]

In [39]:
decoder_target_data=decoder_target_data[:test_length]

In [40]:
encoder_input_data=test_input

In [41]:
print(encoder_input_data.shape)

torch.Size([6880, 24, 768])


In [42]:
decoder_inputs = Input(shape=(None, decoder_input_data.shape[2]))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(decoder_input_data.shape[2], activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [43]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 5504 samples, validate on 1376 samples
Epoch 1/100
5504/5504 [==============================] - 56s 10ms/step - loss: 2.1615 - accuracy: 0.7144 - val_loss: 1.6901 - val_accuracy: 0.7430
Epoch 2/100
5504/5504 [==============================] - 52s 10ms/step - loss: 1.5676 - accuracy: 0.7534 - val_loss: 1.5299 - val_accuracy: 0.7686
Epoch 3/100
5504/5504 [==============================] - 54s 10ms/step - loss: 1.4281 - accuracy: 0.7736 - val_loss: 1.4641 - val_accuracy: 0.7779
Epoch 4/100
5504/5504 [==============================] - 54s 10ms/step - loss: 1.3149 - accuracy: 0.7863 - val_loss: 1.3404 - val_accuracy: 0.7891
Epoch 5/100
5504/5504 [==============================] - 53s 10ms/step - loss: 1.2096 - accuracy: 0.8004 - val_loss: 1.2568 - val_accuracy: 0.8014
Epoch 6/100
5504/5504 [==============================] - 65s 12ms/step - loss: 1.1172 - accuracy: 0.8114 - val_loss: 1.1868 - val_accuracy: 0.8109
Epoch 7/100
5504/5504 [==============================] - 57s 10ms/step

In [44]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [45]:
index_to_word[0]=''

In [46]:
def decode_sequence(input_seq):
    count=0
    states_value = encoder_model.predict(input_seq)
    

    target_seq = np.zeros((1, 1, decoder_input_data.shape[2]))
    target_seq[0, 0, tokenizer.word_index['qwertyuiop']] = 1.
    count += 1

    stop_condition = False
    decoded_sentence = 'qwertyuiop '
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        count += 1

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_word[sampled_token_index]
        decoded_sentence += sampled_char
        decoded_sentence += ' '

        if (sampled_token_index == 0 or
           count > max_len):
            stop_condition = True

        target_seq = np.zeros((1, 1, decoder_input_data.shape[2]))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]
        
    decoded_sentence += '?'

    return decoded_sentence[11:]

In [47]:
print(index_to_word)

{1: 'qwertyuiop', 2: 'the', 3: 'is', 4: 'are', 5: 'a', 6: 'there', 7: 'on', 8: 'in', 9: 'this', 10: 'of', 11: 'man', 12: 'white', 13: '2', 14: '1', 15: 'people', 16: 'does', 17: 'to', 18: 'picture', 19: 'red', 20: 'black', 21: 'person', 22: 'wearing', 23: 'blue', 24: 'holding', 25: 'green', 26: '3', 27: 'have', 28: 'brown', 29: 'cat', 30: 'his', 31: 'dog', 32: 'playing', 33: 'woman', 34: 'and', 35: 'shirt', 36: 'left', 37: 'right', 38: 'photo', 39: 'these', 40: 'they', 41: 'bus', 42: 'made', 43: 'sign', 44: 'yellow', 45: '4', 46: 'train', 47: 'at', 48: 'do', 49: 'sitting', 50: 'table', 51: 'water', 52: 'pizza', 53: "man's", 54: 'for', 55: 'color', 56: 'it', 57: 'you', 58: 'standing', 59: 'hand', 60: 'shown', 61: 'plate', 62: 'animal', 63: 'clock', 64: '0', 65: 'an', 66: 'tennis', 67: 'animals', 68: 'food', 69: 'wall', 70: 'orange', 71: 'room', 72: 'frisbee', 73: 'horse', 74: 'top', 75: 'background', 76: 'can', 77: 'with', 78: 'boy', 79: 'number', 80: 'toilet', 81: '5', 82: 'eating', 83

In [48]:
for seq_index in range(100):
    ipt=train['question'][6880+seq_index]
    ts=val_input[seq_index:seq_index+1]
    dcs=decode_sequence(ts)
    print('Input sentence:', ipt)
    print('Answer:', train['answer'][6880+seq_index])
    print('Decoded sentence:', dcs)

Input sentence: What is the man using?
Answer: skateboard
Decoded sentence: is the man using a skateboard  ?
Input sentence: Why is the hydrant smiling?
Answer: graffiti
Decoded sentence: is the kids being because because of photo  ?
Input sentence: What is the monitor sitting on?
Answer: desk
Decoded sentence: is the cat sitting on the table  ?
Input sentence: What color is the dog?
Answer: black
Decoded sentence: is the dog black  ?
Input sentence: How many horses are present?
Answer: 4
Decoded sentence: are there 4 horses  ?
Input sentence: What is the girl holding?
Answer: donut
Decoded sentence: is the girl holding a pizza  ?
Input sentence: What color is the kite?
Answer: red
Decoded sentence: is the kite red  ?
Input sentence: Are these elephants living in the wild or a zoo?
Answer: wild
Decoded sentence: are these people in this field  ?
Input sentence: How many men are there?
Answer: 3
Decoded sentence: are there 3 men  ?
Input sentence: What is the white stuff on top of the d

Input sentence: What is the elephant statue made from?
Answer: wood
Decoded sentence: is the elephant made of wood  ?
Input sentence: What is the predominant background color featured?
Answer: green
Decoded sentence: is the main color shown in the  ?
Input sentence: What does the sign say?
Answer: stop
Decoded sentence: does the sign say stop  ?
Input sentence: Is the dog standing or laying down?
Answer: standing
Decoded sentence: is the dog standing sitting  ?
Input sentence: What is in the bowl?
Answer: orange
Decoded sentence: is there red in the bowl  ?
Input sentence: How many zebras are in the pic?
Answer: 1
Decoded sentence: is there 1 zebra in the image  ?
Input sentence: How many cats on the couch?
Answer: 2
Decoded sentence: are there 2 cats on the bed  ?
Input sentence: How many computers are there?
Answer: 1
Decoded sentence: is there 1 computer  ?
Input sentence: What sport is being played?
Answer: tennis
Decoded sentence: are they playing tennis  ?
Input sentence: How man